In [1]:
import glob
import numpy as np
import os
import contextlib

In [2]:
import fluidsim

In [3]:
from fluidsim import load_state_phys_file

In [4]:
load_state_phys_file()

IndexError: list index out of range

In [5]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [6]:
params = Simul.create_default_params()
sim = Simul(params)

NameError: name 'Simul' is not defined

In [7]:
from fluidsim import load_state_phys_file

In [8]:
load_state_phys_file('C:\\Users\\sbendl\\Sim_data\\training_data\\NS2D_test_64x64_S10x10_2020-03-03_19-30-11', )

C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\operators\operators2d.py:27: UserWarning: operators2d.py has to be pythranized to be efficient! Install pythran and recompile.
  "operators2d.py has to be pythranized to be efficient! "


*************************************
Program fluidsim
Load state from file:
[...]ning_data\NS2D_test_64x64_S10x10_2020-03-03_19-30-11\state_phys_t010.011.nc
To plot the forcing modes, you can use:
sim.forcing.forcing_maker.plot_forcing_region()
sim:                <class 'fluidsim.solvers.ns2d.solver.Simul'>
sim.oper:           <class 'fluidsim.operators.operators2d.OperatorsPseudoSpectral2D'>
sim.output:         <class 'fluidsim.solvers.ns2d.output.Output'>
sim.state:          <class 'fluidsim.solvers.ns2d.state.StateNS2D'>
sim.time_stepping:  <class 'fluidsim.base.time_stepping.pseudo_spect.TimeSteppingPseudoSpectral'>
sim.init_fields:    <class 'fluidsim.solvers.ns2d.init_fields.InitFieldsNS2D'>
sim.forcing:        <class 'fluidsim.solvers.ns2d.forcing.ForcingNS2D'>

solver NS2D, RK4 and sequential,
type fft: fluidfft.fft2d.with_pyfftw
nx =     64 ; ny =     64
lx = 10 ; ly = 10
path_run =
C:\Users\sbendl\Sim_data\training_data\NS2D_test_64x64_S10x10_2020-03-03_19-30-11
init_fields

C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\base\time_stepping\base.py:108: UserWarning: Cannot handle signals - is multithreading on?
  warn("Cannot handle signals - is multithreading on?")


In [9]:
def make_sim(path, future_steps=10, num_intervals=100):
    with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        sim = load_state_phys_file(path)
    sim_results = [sim.output.phys_fields.get_field_to_plot(key='rot', time=i, interpolate_time=True) for i in np.linspace(0, future_steps, num_intervals)]
    return sim_results

In [11]:
make_sim('C:\\Users\\sbendl\\Sim_data\\training_data\\NS2D_test_64x64_S10x10_2020-03-03_19-30-11')

C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\base\time_stepping\base.py:108: UserWarning: Cannot handle signals - is multithreading on?
  warn("Cannot handle signals - is multithreading on?")


[array([[-0.42328082, -0.28119442, -0.10890705, ..., -0.50953964,
         -0.54707563, -0.51629401],
        [-0.69373077, -0.49025835, -0.2577905 , ..., -0.94733069,
         -0.93424589, -0.84691359],
        [-0.93987503, -0.67872154, -0.39052864, ..., -1.35680304,
         -1.2930616 , -1.15056524],
        ...,
        [ 0.30697448,  0.29190436,  0.30588119, ...,  0.61218108,
          0.46431272,  0.36244613],
        [ 0.10014532,  0.12840865,  0.18677109, ...,  0.30475208,
          0.18340969,  0.11573266],
        [-0.15078465, -0.06873393,  0.04386869, ..., -0.08000585,
         -0.16364046, -0.18597682]]),
 array([[-0.81650874, -0.68360135, -0.4665529 , ..., -0.57170525,
         -0.76133609, -0.84500392],
        [-1.03340502, -0.85612363, -0.60055824, ..., -0.95054345,
         -1.08290599, -1.11193151],
        [-1.22212401, -0.99995391, -0.70739157, ..., -1.31263465,
         -1.38266495, -1.3530818 ],
        ...,
        [-0.16028438, -0.12672688, -0.00723334, ...,  

In [12]:
import os

In [13]:
def sim_streamer(path):
    while True:
        for sim in os.listdir(path):
            sim_results = make_sim(path + '\\' + sim)
            for step in range(len(sim_results) - 1):
                yield sim_results[step], sim_results[step+1]

In [14]:
streamer = sim_streamer(r'C:\Users\sbendl\Sim_data\training_data')

In [15]:
np.stack(list(islice(streamer, 4)))[:,0,:,:].shape

NameError: name 'islice' is not defined

In [16]:
from itertools import islice
def batch_generator(path, batch_size=16):
    streamer = sim_streamer(path)
    while True:
        batch = np.stack(list(islice(streamer, batch_size)))
        yield batch[:,0,:,:].reshape(16, 64, 64, 1),batch[:,1,:,:].reshape(16, 64, 64, 1)


In [17]:
gen = batch_generator(r'C:\Users\sbendl\Sim_data\training_data')

In [18]:
next(gen)[1].shape

C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\base\time_stepping\base.py:108: UserWarning: Cannot handle signals - is multithreading on?
  warn("Cannot handle signals - is multithreading on?")


(16, 64, 64, 1)

In [19]:
import keras.backend as K
from keras.layers import *

Using TensorFlow backend.


In [20]:
from keras.models import Sequential

In [21]:
K.clear_session()
m = Sequential()

m.add(Conv2D(16, 4, input_shape=(64,64,1), padding='same', data_format='channels_last'))
m.add(Conv2D(16, 4, padding='same', data_format='channels_last'))
m.add(Conv2D(1, 4, padding='same', data_format='channels_last'))

m.compile(optimizer='adam', loss='mse')
m.build()
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        4112      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 1)         257       
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
_________________________________________________________________


In [22]:
hist = m.fit_generator(gen, steps_per_epoch=128, epochs=16, validation_steps=128)

Epoch 1/16


C:\Users\sbendl\AppData\Local\Continuum\anaconda3\lib\site-packages\fluidsim\base\time_stepping\base.py:108: UserWarning: Cannot handle signals - is multithreading on?
  warn("Cannot handle signals - is multithreading on?")


128/128 [==============================] - 14s 106ms/step - loss: 0.9273
Epoch 2/16
128/128 [==============================] - 13s 101ms/step - loss: 0.5018
Epoch 3/16
128/128 [==============================] - 13s 100ms/step - loss: 0.4669
Epoch 4/16
128/128 [==============================] - 13s 102ms/step - loss: 0.4594
Epoch 5/16
128/128 [==============================] - 13s 102ms/step - loss: 0.4657
Epoch 6/16
128/128 [==============================] - 13s 100ms/step - loss: 0.4351
Epoch 7/16
128/128 [==============================] - 13s 102ms/step - loss: 0.44421s
Epoch 8/16
128/128 [==============================] - 13s 102ms/step - loss: 0.4109
Epoch 9/16
128/128 [==============================] - 13s 101ms/step - loss: 0.4219
Epoch 10/16
128/128 [==============================] - 13s 100ms/step - loss: 0.4249
Epoch 11/16
128/128 [==============================] - 13s 102ms/step - loss: 0.43290s - los
Epoch 12/16
128/128 [==============================] - 13s 101ms/step - los

In [ ]:
hist